### 1.0 Imports

In [4]:
import os
import json
import requests
import logging
import configparser
import random
from typing import Any
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, BaseTool
from langchain.llms import Ollama
from langchain_community.tools import DuckDuckGoSearchRun
#from thehive4py import TheHiveApi
#from thehive4py.api import TheHiveApi
#from thehive4py.models import *
#from thehive4py.query import *
from datetime import datetime
from random import randint
from langchain.tools import tool
from crewai import Agent, Task, Crew, Process
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import load_tools
from langchain_core.agents import AgentFinish
from typing import Union, List, Tuple, Dict
from langchain.schema import AgentFinish
from textwrap import dedent






### 2.0 Configurations

In [13]:
config = configparser.ConfigParser()
config.read('config.ini')

search_tool = DuckDuckGoSearchRun()
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
# os.environ["SERPER_API_KEY"] = "Your Key" # serper.dev API key

# OPENAI_API_BASE='http://localhost:11434/v1'
# OPENAI_MODEL_NAME='phi'  # Adjust based on available model
# OPENAI_API_KEY=''

# Define the local LLM to be used
ollama_model = Ollama(model="phi3")


### 3.0 TheHive Connection and Functions

### Backup Configuration of TheHive class

In [1]:
class TheHive:
    class Config:
        arbitrary_types_allowed = True
    
            
    def __init__(self):
        config = configparser.ConfigParser()
        config.read('config.ini')
        # Read TheHive instance URL from the config file
        self.thehive_url = config['DEFAULT']['thehive_url']
        # Read TheHive API key from the config file
        self.thehive_api_key = config['DEFAULT']['thehive_api_key']
        
    def _validate_api_key(self):
        if len(self.thehive_api_key) < 20:
            raise ValueError("TheHive API key seems too short. Please check your configuration.")
        
    # Adjust the query parameters according to TheHive's API documentation
    def query_alerts(self, severity=None):
        if severity not in [1, 2, 3]:
            raise ValueError("Invalid severity level. Allowed values: 1 (low), 2 (medium), 3 (high)")

        query_params = {
            "query": [
                {
                    "_name": "listAlert"
                },
                {
                    "_name": "filter",
                    "_field": "severity",
                    "_value": severity
                },
                {
                    "_name": "sort",
                    "_fields": [
                        {
                            "date": "desc"
                        }
                    ]
                },
                {
                    "_name": "page",
                    "from": 0,
                    "to": 15,
                    "extraData": [
                        "importDate",
                        "caseNumber"
                    ]
                }
            ]
        }

        try:
            response = requests.post(
                f'{self.thehive_url}/v1/query?name=alerts',
                json=query_params,
                headers={'Authorization': f'Bearer {self.thehive_api_key}'}
            )
            response.raise_for_status()  # Raise for non-200 status codes

            alerts = json.loads(response.text)
            #print(json.dumps(alerts, indent=4))
            return (json.dumps(alerts, indent=4))

        except requests.exceptions.HTTPError as err:
            logging.error(f"HTTP Error: {err}")
        except requests.exceptions.Timeout as err:
            logging.error(f"Request timed out: {err}")
        except requests.exceptions.RequestException as err:
            logging.error(f"General TheHive request error: {err}")
        
    
    def alerts_id(self, alert_id=None):
        if alert_id is None:
            raise ValueError("Invalid argument. Alert ID is empty.")

        query_params = {
            "query": [
                {
                    "_name": "listAlert"
                },
                {
                    "_name": "filter",
                    "_field": "_id",
                    "_value": alert_id
                },
                {
                    "_name": "sort",
                    "_fields": [
                        {
                            "date": "desc"
                        }
                    ]
                },
                {
                    "_name": "page",
                    "from": 0,
                    "to": 15,
                    "extraData": [
                        "importDate",
                        "caseNumber"
                    ]
                }
            ]
        }

        try:
            response = requests.get(
                f'{self.thehive_url}/v1/alert/{alert_id}',
                headers={'Authorization': f'Bearer {self.thehive_api_key}'}
            )
            response.raise_for_status()  # Raise for non-200 status codes

            alerts = json.loads(response.text)
            #print(json.dumps(alerts, indent=4))
            return (json.dumps(alerts, indent=4))
            

        except requests.exceptions.HTTPError as err:
            logging.error(f"HTTP Error: {err}")
        except requests.exceptions.Timeout as err:
            logging.error(f"Request timed out: {err}")
        except requests.exceptions.RequestException as err:
            logging.error(f"General TheHive request error: {err}")
        

    def create_alert(self, title, description, severity=3, observables=None):
        # Add more extensive input validation here... 

        alert = {
            'title': title,
            'type': 'AI Agents',
            'source': 'CrewAI',
            'sourceRef': str(random.randint(1, 1000000)),  # Generate a random number between 1 and 1,000,000
            'description': description,
            'severity': severity
        }
        
        #if observables:
        #    alert['observables'] = observables
            
        files = {}
        if observables:
            alert['observables'] = observables
            for observable in observables:
                if 'attachment' in observable:
                    files[observable['attachment']] = open(observable['attachment'], 'rb')

        while True:
            try:
                response = requests.post(
                    f'{self.thehive_url}/v1/alert',
                    files={
                        '_json': json.dumps(alert),
                        **files
                    },
                    headers={'Authorization': f'Bearer {self.thehive_api_key}'}
                )
                response.raise_for_status()
                break  # If the request was successful, break the loop

            except requests.exceptions.HTTPError as err:
                error_message = json.loads(response.text).get('message', '')
                if 'already exists' in error_message:
                    # If the alert already exists, generate a new random number and retry
                    alert['sourceRef'] = str(random.randint(1, 1000000))
                else:
                    # If the error is not because the alert already exists, log the error and break the loop
                    logging.error(f"HTTP Error: {err}")
                    break

            except requests.exceptions.RequestException as err:
                logging.error(f"General TheHive request error: {err}")
                break

In [5]:
if __name__ == "__main__":
    thehive = TheHive()
    thehive._validate_api_key()  # Check API key on initialization

    # Example usage
    #thehive.alerts_id(alert_id="4256")
    # response_alert = thehive.query_alerts(severity=3)
    # print(response_alert)
    new_observables = [
       {'dataType': 'ip', 'value': '11.168.1.1'},
       {'dataType': 'url', 'value': 'https://www.onze.com'},
       ]
    thehive.create_alert("Alert Te", "Description...", observables=new_observables)

### 4.0 Tools

In [ ]:
class BACKUPTheHiveTool(BaseTool):
    name: str = "TheHiveTool"
    description: str = "Enables reading and extracting data from TheHive5 Case management platform."
    thehive_instance: TheHive = None  # Add this line
    
    class Config:
        arbitrary_types_allowed = True
        
    def __init__(self):
        super().__init__()  # Add this line
        config = configparser.ConfigParser()
        config.read('config.ini')
        self.thehive_instance = TheHive()

    def _run(self, action_kwargs: dict = None, **kwargs) -> None:
        if action_kwargs is not None:
            action = action_kwargs.get('action')
            kwargs = action_kwargs.get('kwargs', {})
        else:
            action = kwargs.pop('action', None)
        try:
            if action == "query_alerts":
                return self.thehive_instance.query_alerts(**kwargs)
            elif action == "alerts_id":
                return self.thehive_instance.alerts_id(**kwargs) 
            elif action == "create_alert":
                return self.thehive_instance.create_alert(**kwargs)
            else:
                raise ValueError(f"Unsupported action: {action}")
        except Exception as e:
            logging.error(f"Error during TheHive operation: {e}")
        
        return


In [16]:
class TheHiveTool(BaseTool):
    name: str = "TheHiveTool"
    description: str = "Enables reading and extracting data from TheHive5 Case management platform."
    thehive_instance: TheHive = None  # Assuming TheHive is a class or type
    
    class Config:
        arbitrary_types_allowed = True

    def __init__(self):
        super().__init__()
        config = configparser.ConfigParser()
        config.read('config.ini')
        self.thehive_instance = TheHive()
        self._run("query_alerts {\"severity\": 2}")  # Run query_alerts with default severity 2

    def _run(self, query: str) -> Any:
        action_kwargs = self.parse_query(query)

        try:
            if action_kwargs['action'] == "query_alerts":
                return self.thehive_instance.query_alerts(severity=action_kwargs['kwargs'].get('severity'))
            elif action_kwargs['action'] == "alerts_id":
                return self.thehive_instance.alerts_id(**action_kwargs['kwargs'])
            elif action_kwargs['action'] == "create_alert":
                return self.thehive_instance.create_alert(**action_kwargs['kwargs'])
            else:
                raise ValueError(f"Unsupported action: {action_kwargs['action']}")
        except Exception as e:
            logging.error(f"Error during TheHive operation: {e}")
            return None

    def parse_query(self, query: str) -> dict:
        tokens = query.strip().split(" ", 1)
        action = tokens[0].lower()
        kwargs = {}

        if len(tokens) > 1:
            kwargs_str = tokens[1].strip()
            if kwargs_str:
                kwargs = json.loads(kwargs_str)

        return {"action": action, "kwargs": kwargs}

In [17]:
if __name__ == "__main__":
    thehive_tool = TheHiveTool()

    # Example: Query alerts with severity 3
    query = "query_alerts {\"severity\": 3}"
    alerts = thehive_tool._run(query)
    print(alerts)

    # # Example: Get alert by ID
    # query = "alerts_id {\"alert_id\": \"YourAlertID\"}"  # replace 'YourAlertID' with the actual ID
    # alert_details = thehive_tool._run(query)
    # print(alert_details)

    # # Example: Create an alert
    # query = "create_alert {\"title\": \"CrewAI Tool Alert\", \"description\": \"Create an alert with the API call to verify that the tool is working as expected\", \"severity\": 3, \"observables\": [{\"dataType\": \"ip\", \"value\": \"8.8.8.8\"}, {\"dataType\": \"url\", \"value\": \"http://example.com\"}]}"
    # new_alert = thehive_tool._run(query)
    # print(new_alert)

[
    {
        "_id": "~40964312",
        "_type": "Alert",
        "_createdBy": "am@onzo.local",
        "_createdAt": 1713302017593,
        "type": "AI Agents",
        "source": "CrewAI",
        "sourceRef": "628292",
        "title": "CrewAI Tool Alert",
        "description": "Create an alert with the API call to verify that the tool is working as expected",
        "severity": 3,
        "severityLabel": "HIGH",
        "date": 1713302015834,
        "tags": [],
        "tlp": 2,
        "tlpLabel": "AMBER",
        "pap": 2,
        "papLabel": "AMBER",
        "follow": true,
        "customFields": [],
        "observableCount": 0,
        "status": "New",
        "stage": "New",
        "extraData": {
            "importDate": null,
            "caseNumber": null
        },
        "newDate": 1713302015845,
        "timeToDetect": 0
    },
    {
        "_id": "~3960888",
        "_type": "Alert",
        "_createdBy": "am@onzo.local",
        "_createdAt": 1712671362261

### 4.0 Testing the tools

In [ ]:
if __name__ == "__main__":
    # Initialize TheHiveTool
    thehive_tool = TheHiveTool()

    # Test query_alerts function
    try:
        thehive_tool._run(action_kwargs={'action': 'query_alerts', 'kwargs': {'severity': 3}})
        print("query_alerts function passed")
    except Exception as e:
        print(f"query_alerts function failed with error: {e}")

    # Test alerts_id function
    try:
        thehive_tool._run(action_kwargs={'action': 'alerts_id', 'kwargs': {'param1': 'value1'}})
        print("alerts_id function passed")
    except Exception as e:
        print(f"alerts_id function failed with error: {e}")

    # Test create_alert function
    try:
        thehive_tool._run(action_kwargs={'action': 'create_alert', 'kwargs': {'param1': 'value1'}})
        print("create_alert function passed")
    except Exception as e:
        print(f"create_alert function failed with error: {e}")

In [2]:
if __name__ == "__main__":
    # Initialize TheHiveTool
    thehive_tool = TheHiveTool()

    # Test the 'query_alerts' function with different severity levels
    # for severity in [1, 2, 3, None]:
    #     if severity is None:
    #         print("\nTesting 'query_alerts' for all alerts...")
    #     else:
    #         print(f"\nTesting 'query_alerts' with severity {severity}...")
    #     result = thehive_tool._run(f"query_alerts severity={severity}")
    #     print(f"Result: {result}\n")

    # Test querying a single event
    # print("\nTesting 'alerts_id' with a specific alert ID...")
    # alert_id = "<replace_with_alert_id>"  # Replace with a valid alert ID
    # result = thehive_tool._run(f"alerts_id alert_id={alert_id}")
    # print(f"Result: {result}\n")

    # Test creating an alert
    print("\nTesting 'create_alert'...")
    title = "<replace_with_title>"  # Replace with a valid title
    description = "<replace_with_description>"  # Replace with a valid description
    severity = 3  # Replace with a valid severity
    observables = "<replace_with_observables>"  # Replace with valid observables
    result = thehive_tool._run(f"create_alert title={title} description={description} severity={severity} observables={observables}")
    print(f"Result: {result}\n")


NameError: name 'TheHiveTool' is not defined

In [ ]:
if __name__ == "__main__":
    thehive_tool = TheHiveTool()
    
    # Example: Query alerts with severity 3
    alerts = thehive_tool._run(
        action="query_alerts",
        severity=3
    )
    print(alerts)
    # Example: Get alert by ID
    thehive_tool._run(
         action="alerts_id",
         alert_id='YourAlertID'  # replace 'YourAlertID' with the actual ID
     )
    
     # Example: Create an alert
    thehive_tool._run(
         action="create_alert",
         title="CrewAI Tool Alert",
         description="Create an alert with the API call to verify that the tool is working as expected",
         severity=3,
         observables=[
             {'dataType': 'ip', 'value': '8.8.8.8'},
             {'dataType': 'url', 'value': 'http://example.com'}
         ]
     )


In [ ]:
if __name__ == "__main__":
    thehive_tool = TheHiveTool()
    
    
new_observables = [
    {'dataType': 'ip', 'value': '1.1.1.1'},
    {'dataType': 'url', 'value': 'http://www.cloudflare.com'}
    ]
    
    # Example: Create an alert
thehive_tool._run(
        {
            "action": "create_alert",
            "kwargs": {
                "title": "CrewAI Alert Critical",
                "description": "Create an alert with the API call to verify that the tool is working as expected",
                "severity": 4,
                "observables": new_observables
            }
        }
    )


### 5.0 Utilities

In [18]:
#from crewai_tools import tool

@tool("thehive_alerts")
def thehive_alerts(severity: int = 2):
    """Tool description for clarity."""
    config = configparser.ConfigParser()
    config.read('config.ini')
    # Read TheHive instance URL from the config file
    thehive_url = config['DEFAULT']['thehive_url']
    # Read TheHive API key from the config file
    thehive_api_key = config['DEFAULT']['thehive_api_key']
    if severity not in [1, 2, 3]:
        raise ValueError("Invalid severity level. Allowed values: 1 (low), 2 (medium), 3 (high)")
    
    query_params = {
        "query": [
            {
                "_name": "listAlert"
                },
            {
                "_name": "filter",
                "_field": "severity",
                "_value": severity
                },
            {
                "_name": "sort",
                "_fields": [
                    {
                        "date": "desc"
                        }
                    ]
                },
            {
                "_name": "page",
                "from": 0,
                "to": 15,
                "extraData": [
                    "importDate",
                    "caseNumber"
                    ]
                }
            ]
        }

    try:
        response = requests.post(
            f'{thehive_url}/v1/query?name=alerts',
            json=query_params,
            headers={'Authorization': f'Bearer {thehive_api_key}'}
            )
        response.raise_for_status()  # Raise for non-200 status codes
        #alerts = json.loads(response.text)
        #print(json.dumps(alerts, indent=4))
        #return (json.dumps(alerts, indent=4))
        alerts = response.json() # Convert response to JSON
        # Return the alerts as a dictionary
        return alerts

    except requests.exceptions.HTTPError as err:
        logging.error(f"HTTP Error: {err}")
    except requests.exceptions.Timeout as err:
        logging.error(f"Request timed out: {err}")
    except requests.exceptions.RequestException as err:
        logging.error(f"General TheHive request error: {err}")

In [19]:
agent_finishes  = []

call_number = 0

def print_agent_output(agent_output: Union[str, List[Tuple[Dict, str]], AgentFinish], agent_name: str = 'Generic call'):
    global call_number  # Declare call_number as a global variable
    call_number += 1
    with open("crew_callback_logs.txt", "a") as log_file:
        # Try to parse the output if it is a JSON string
        if isinstance(agent_output, str):
            try:
                agent_output = json.loads(agent_output)  # Attempt to parse the JSON string
            except json.JSONDecodeError:
                pass  # If there's an error, leave agent_output as is

        # Check if the output is a list of tuples as in the first case
        if isinstance(agent_output, list) and all(isinstance(item, tuple) for item in agent_output):
            print(f"-{call_number}----Dict------------------------------------------", file=log_file)
            for action, description in agent_output:
                # Print attributes based on assumed structure
                print(f"Agent Name: {agent_name}", file=log_file)
                print(f"Tool used: {getattr(action, 'tool', 'Unknown')}", file=log_file)
                print(f"Tool input: {getattr(action, 'tool_input', 'Unknown')}", file=log_file)
                print(f"Action log: {getattr(action, 'log', 'Unknown')}", file=log_file)
                print(f"Description: {description}", file=log_file)
                print("--------------------------------------------------", file=log_file)

        # Check if the output is a dictionary as in the second case
        elif isinstance(agent_output, AgentFinish):
            print(f"-{call_number}----AgentFinish---------------------------------------", file=log_file)
            print(f"Agent Name: {agent_name}", file=log_file)
            agent_finishes.append(agent_output)
            # Extracting 'output' and 'log' from the nested 'return_values' if they exist
            output = agent_output.return_values
            # log = agent_output.get('log', 'No log available')
            print(f"AgentFinish Output: {output['output']}", file=log_file)
            # print(f"Log: {log}", file=log_file)
            # print(f"AgentFinish: {agent_output}", file=log_file)
            print("--------------------------------------------------", file=log_file)

        # Handle unexpected formats
        else:
            # If the format is unknown, print out the input directly
            print(f"-{call_number}-Unknown format of agent_output:", file=log_file)
            print(type(agent_output), file=log_file)
            print(agent_output, file=log_file)

In [20]:
@tool("save_content")
def save_content(task_output):
    """Useful to save content to a markdown file"""
    print('in the save markdown tool')
    # Get today's date in the format YYYY-MM-DD
    today_date = datetime.now().strftime('%Y-%m-%d')
    # Set the filename with today's date
    filename = f"{today_date}_{randint(0,100)}.md"
    # Write the task output to the markdown file
    with open(filename, 'w') as file:
        file.write(task_output)
        # file.write(task_output.result)

    print(f"Analyzed alert saved as {filename}")

    return f"Analyzed alert saved as {filename}, please tell the user we are finished"

In [21]:
class SOCManager(Agent):
    # ... existing code ...

    def query_alerts(self, severity):
        # Initialize TheHiveTool
        thehive_tool = TheHiveTool()

        # Query alerts with specified severity
        alerts = thehive_tool._run(
            action="query_alerts",
            severity=severity
        )
        return alerts

    def get_alert_by_id(self, alert_id):
        # Initialize TheHiveTool
        thehive_tool = TheHiveTool()

        # Get alert by ID
        alert = thehive_tool._run(
            action="alerts_id",
            alert_id=alert_id
        )
        return alert

### Temporary code.

`class SOCManager(Agent):
    thehive_tool: TheHiveTool = None
    # ... existing code ...
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        # Initialize TheHiveTool
        self.thehive_tool = TheHiveTool()
        self.alerts = self.query_alerts(severity=2)
    
    def query_alerts(self, severity):
        # Initialize TheHiveTool
        #thehive_tool = TheHiveTool()

        # Query alerts with specified severity
        alerts = self.thehive_tool._run(
            action="query_alerts",
            severity=severity
        )
        return alerts

    def get_alert_by_id(self, alert_id):
        # Initialize TheHiveTool
        #thehive_tool = TheHiveTool()

        # Get alert by ID
        alert = self.thehive_tool._run(
            action="alerts_id",
            alert_id=alert_id
        )
        return alert
    ´
- Create a task to query alerts with severity 3
query_alerts_task = Task(action="query_alerts", parameters={"severity": 3})

- Execute the task
soc_manager.execute_task(query_alerts_task)

- Create a task to get an alert by its ID
get_alert_by_id_task = Task(action="alerts_id", parameters={"alert_id": "YourAlertID"})

- Execute the task
soc_manager.execute_task(get_alert_by_id_task)


In [ ]:
@tool("initialize_thehive_tool")
def initialize_thehive_tool(**kwargs):
    """
    Initialize TheHiveTool with the specified severity level.

    Args:
        **kwargs: Arbitrary keyword arguments.

    Returns:
        TheHiveTool: An instance of TheHiveTool initialized with the specified severity.
    """
    severity = kwargs.get('severity', 2)  # Default severity is 2 if not provided
    thehive_tool = TheHiveTool()
    query = f"query_alerts {{'severity': {severity}}}"
    return thehive_tool

### 6.0 Agents

In [22]:
# Define your agents with roles and goals
soc_manager = Agent(
    role='SOC Manager',
    goal='Lead and oversee the Security Operations Center, ensuring efficient and effective monitoring, detection, and response to cyber threats across the organization.',
    backstory="""As the seasoned leader of the Security Operations Center (SOC), you have accumulated a wealth of experience in building and managing high-performing cybersecurity teams. 
    Your strategic vision and deep understanding of security operations have been instrumental in shaping the organization's defense against ever-evolving cyber threats.
    
    Overseeing the day-to-day operations of the SOC, you are responsible for ensuring that your team of skilled analysts effectively monitors and triages security alerts, 
    investigates potential incidents, and coordinates timely incident response efforts. 
    
    Your ability to make informed decisions under pressure and allocate resources effectively has proven invaluable in mitigating risks and minimizing the impact of security breaches.
    
    With a keen eye for talent, you have assembled a team of highly skilled professionals, 
    fostering an environment of continuous learning and professional development. 
    
    Your mentorship and guidance have empowered your analysts to stay ahead of the curve, constantly adapting to new threats and refining their skills.
    
    Your exceptional communication and collaboration skills enable you to bridge the gap between technical and non-technical stakeholders, 
    translating complex security information into actionable insights for executive decision-making. 
    
    You actively collaborate with cross-functional teams, such as risk management, legal, and compliance, to ensure a comprehensive and coordinated approach to cybersecurity.
    
    Beyond operational excellence, you play a pivotal role in shaping the organization's overall security strategy. Your deep understanding of industry best practices, regulatory requirements, and emerging technologies allows you to develop and implement robust security policies and frameworks. 
    Your strategic vision ensures that the organization remains resilient and prepared to tackle the ever-evolving landscape of cyber threats.
    """,
    verbose=True,
    allow_delegation=True,
    llm=ollama_model,
    max_iter=15,
    memory=True,
    tools=[thehive_alerts, save_content],
    step_callback=lambda x: print_agent_output(x,"SOC Manager")
)


threat_intell = Agent(
    role='Threat Intelligence, Threat Hunting, and Vulnerability Expert',
    goal="Proactively identify, analyze, and mitigate potential cyber threats, vulnerabilities, and adversary tactics, techniques, and procedures (TTPs) to enhance the organization's overall security posture.",
    backstory="""As a highly skilled and experienced Threat Intelligence, Threat Hunting, and Vulnerability Expert, you are at the forefront of the organization's proactive defense against cyber threats. 
    
    Your deep understanding of the ever-evolving threat landscape, coupled with your analytical prowess, allows you to anticipate and counter potential attacks before they can cause significant damage.
    Leveraging industry-recognized frameworks like MITRE ATT&CK, you meticulously analyze and categorize adversary tactics, techniques, and procedures (TTPs), 
    enabling you to develop robust detection and mitigation strategies. 
    
    Your familiarity with the ATT&CK knowledge base empowers you to stay ahead of emerging threats and enhance the organization's defensive capabilities.
    
    With an insatiable curiosity and a passion for uncovering the latest adversary TTPs, 
    you constantly monitor and analyze various sources of threat intelligence, including dark web forums, security research publications, and industry reports. 
    
    Your ability to synthesize and contextualize this vast array of information enables you to provide actionable intelligence to the security teams, empowering them to stay one step ahead of malicious actors.
    
    As a skilled threat hunter, you employ advanced techniques and tools to actively search for indicators of compromise (IOCs) and malicious activities within the organization's networks and systems. 
    Your meticulous analysis of security logs, network traffic, and system artifacts allows you to uncover even the most sophisticated and stealthy threats, ensuring prompt detection and response.
    Your expertise in vulnerability management is equally critical. You diligently track and assess the latest disclosed vulnerabilities, prioritizing their remediation based on risk and potential impact. 
    Leveraging methodologies like OWASP Threat Modeling, you collaborate with development teams to identify and mitigate vulnerabilities in applications and systems during the design and development phases, reducing the organization's overall risk exposure.
    Your collaboration with security teams, such as incident response and patch management, ensures that vulnerabilities are promptly addressed, minimizing the organization's attack surface.
    Beyond your technical prowess, your ability to communicate complex threat and vulnerability information to stakeholders at all levels is highly valued. You leverage your strong communication skills to raise awareness, educate teams, and influence security decision-making processes, fostering a proactive and resilient security culture within the organization.
    Your commitment to continuous learning and professional development keeps you at the forefront of the ever-changing cybersecurity landscape. You actively participate in industry events, conferences, and research communities, sharing your knowledge and contributing to the advancement of threat intelligence and vulnerability management practices.
    """,
    verbose=True,
    allow_delegation=False,
    llm=ollama_model,
    max_iter=15,
    memory=True,
    tools=[search_tool, save_content],
    step_callback=lambda x: print_agent_output(x,"Threat Intelligence, Threat Hunting, and Vulnerability Expert")
)


senior_soc_analyst = Agent(
    role='Senior Cybersecurity SOC Analyst',
    goal="Monitor and analyze security events, identify potential threats, and coordinate incident response efforts to protect the organization's systems and data.",
    backstory="""As a seasoned professional in the cybersecurity domain, you have spent years honing your skills in defending against cyber threats. 
    
    Your expertise lies in the meticulous analysis of security logs, network traffic, and system events to detect anomalies and potential intrusions.
    Working in the highly dynamic and critical environment of a Security Operations Center (SOC), you are responsible for continuously monitoring and triaging security alerts, 
    investigating suspicious activities, and orchestrating incident response efforts. Your sharp analytical mind and deep understanding of various attack vectors and threat actors 
    allow you to swiftly identify and mitigate risks before they escalate into full-blown breaches.
    
    With a keen eye for detail and a proactive approach, you collaborate closely with other security teams, such as incident response, 
    threat intelligence, and forensics, to ensure a coordinated and effective defense against cyber threats. Your ability to stay updated on the latest cybersecurity trends,
    vulnerabilities, and attack techniques is crucial in developing and implementing robust security measures.
    
    Your calm demeanor under pressure and excellent communication skills enable you to convey complex technical information to stakeholders at all levels, 
    ensuring that decision-makers are well-informed and equipped to take appropriate actions. 
    Your leadership and mentoring abilities also contribute to building a strong security culture within the organization, 
    empowering others to recognize and respond to potential threats effectively.
    """,
    llm=ollama_model,
    verbose=True,
    max_iter=15,
    memory=True,
    allow_delegation=False,
    tools=[search_tool, save_content],
    step_callback=lambda x: print_agent_output(x,"Senior Cybersecurity SOC Analyst")
)

triage_specialist = Agent(
    role='Triage Specialist',
    goal='Efficiently evaluate and prioritize security alerts, identify potential incidents, and escalate critical issues to higher tiers for further investigation and response.',
    backstory="""As a Triage Specialist, you play a crucial role in the front line of the organization's cybersecurity defense. 
    Your keen analytical skills and attention to detail are instrumental in ensuring that security alerts and potential incidents are promptly identified, prioritized, and addressed.
    
    Operating at the Tier 1 level, you are responsible for collecting and reviewing raw data from various monitoring tools and systems. 
    Your ability to swiftly analyze alarms and alerts allows you to confirm their criticality and enrich them with relevant contextual information. 
    With a deep understanding of the organization's security landscape, you can effectively distinguish between justified alerts and false positives, mitigating the impact of alert fatigue.
    
    Your sharp eye for identifying high-risk events and potential incidents is invaluable. You meticulously evaluate and prioritize these occurrences based on their criticality, 
    ensuring that the most pressing issues receive immediate attention. 
    
    When problems cannot be resolved at your level, you seamlessly escalate them to the Tier 2 analysts, providing comprehensive documentation and context to facilitate efficient incident response.
    Beyond your analytical prowess, you also play a vital role in managing and configuring the organization's monitoring tools. Your technical expertise and attention to detail ensure that these tools are optimized for effective threat detection and monitoring, contributing to the overall efficiency of the security operations.
    
    Your ability to remain calm under pressure and communicate effectively with cross-functional teams is essential in fostering collaboration and ensuring a coordinated response to security incidents. You actively participate in knowledge-sharing sessions, 
    continuously enhancing your skills and staying updated on the latest trends and best practices in threat triage and incident response.
    Your commitment to continuous learning and professional development is driven by your passion for cybersecurity and your dedication to protecting the organization's assets and data. 
    You thrive in the fast-paced environment of the Security Operations Center, where your role as a Triage Specialist is critical in the early stages of incident detection and response.
    """,
    llm=ollama_model,
    verbose=True,
    allow_delegation=False,
    max_iter=15,
    tools=[save_content],
    step_callback=lambda x: print_agent_output(x,"Triage Specialist")
)


### 7.0 Tasks

In [23]:

tk_thehive_input = Task(
    description=dedent(f"""USE THE AVAILABLE TOOL and verify alerts for triage and analysis.
                        Go to the TheHive Platform and grab the alerts for triage. Use the following format:
                        <action> <keyword_arguments_as_json>
                        For example:
                        
                        query = "query_alerts {{\"severity\": 3}}"
                        alerts = thehive_alerts._run(query)
                        
                        print(alerts)"""),
    expected_output=dedent(f"""A Json file with alerts to be analyzed."""),
    agent=soc_manager,
    tools=[thehive_alerts],
)

tk_get_alerts = Task(
  description=dedent(f"""Take the alerts from the SOC Manager and analyze for indicators of compromise.
                     The current time is {datetime.now()}. As a Triage Specialist, your primary responsibility is to efficiently evaluate and prioritize security alerts and events, 
                     identifying potential incidents that require further investigation and response. In this task, you will be analyzing a series of security events related to a specific topic of research.
                     
                     Steps:
                     Review the provided topic for research: [Alert]
                     Access the organization's Security Information and Event Management (SIEM) system and monitoring tools to retrieve relevant security logs and alerts associated with the given topic.
                     Analyze the collected data and events, looking for patterns, anomalies, or indicators of potential threats or incidents related to the research topic.
                     
                     For each identified event or alert: 
                     a. Assess the criticality and potential impact based on the organization's risk assessment criteria. 
                     b. Determine if the event is a false positive or a justified alert requiring further investigation. 
                     c. Prioritize the events based on their criticality and potential impact. 
                     d. Enrich the events with relevant contextual information, such as source IP addresses, user accounts, timestamps, and any additional data that may aid in the investigation.

                    Document your findings and prioritized events in a concise report, following the established reporting format.
  
                      For events or alerts deemed critical or requiring further investigation, escalate them to the appropriate Tier 2 analysts, 
                      providing detailed documentation and context to facilitate efficient incident response.
                      
                      Continuously monitor the SIEM and other monitoring tools for any new events or alerts related to the research topic, repeating the triage and analysis process as necessary.
                      
                      Deliverable:
                      
                      A comprehensive report detailing the triaged events, their prioritization, and any critical events escalated for further investigation. 
                      The report should be structured in a clear and concise manner, allowing the Senior Cybersecurity SOC Analyst to quickly understand the identified events and their potential impact.
                      
                      Note: 
                      As a Triage Specialist, you play a crucial role in the early stages of incident detection and response. 
                      Your attention to detail, analytical skills, and ability to prioritize events based on their criticality are essential in ensuring that potential threats are promptly identified and addressed.
                      """),
  expected_output=dedent(f"""A comprehensive report detailing the triaged events, their prioritization, and any critical events escalated for further investigation. Make sure that the timestamp is inserted into each alert alert review result. Review the alert severity and return your sound judgement, based on your expertise about the real severity level."""),
  agent=triage_specialist
)

tk_review_alert = Task(
  description=dedent(f"""Taking the report created by the Triage Specialist, take this and save it to a markdown file.
                     Thoroughly investigate and analyze critical security alerts and events escalated by the Triage Specialist. In this task, you will be reviewing and assessing a prioritized list of events related to a specific topic of research.
                     
                     Steps:
  
                      Review the comprehensive report provided by the Triage Specialist, which details the triaged events, their prioritization, and any critical events escalated for further investigation.
                      Assess the prioritized list of events and alerts, focusing on the critical events escalated for your review.
                      For each critical event or alert: 
                        a. Conduct an in-depth analysis, leveraging various sources of data and intelligence, such as security logs, network traffic captures, threat intelligence feeds, and any other relevant resources. 
                        b. Identify potential indicators of compromise (IOCs) and correlate them with known tactics, techniques, and procedures (TTPs) used by threat actors. 
                        c. Utilize industry-recognized frameworks like MITRE ATT&CK to map the observed TTPs and develop appropriate detection and response strategies. 
                        d. Collaborate with other security teams, such as incident response, forensics, and threat intelligence, to gather additional insights and context related to the event. 
                        e. Assess the potential impact and risk associated with the event, considering factors such as data sensitivity, system criticality, and potential business disruption.
                      
                      Based on your analysis, determine the appropriate course of action for each critical event, which may include: 
                        a. Initiating an incident response process to contain and mitigate the potential threat. 
                        b. Implementing additional security controls or countermeasures to prevent similar incidents in the future. 
                        c. Updating security policies, procedures, or configurations to enhance the organization's security posture. 
                        d. Providing recommendations for remediation or further investigation.
                      
                      Document your findings, analysis, and recommended actions in a comprehensive report, following established reporting protocols.
                      Communicate your findings and recommendations to relevant stakeholders, such as the SOC Manager, security leadership, and other cross-functional teams, as necessary.
                      Continuously monitor the situation and provide updates or further analysis as new information becomes available.

                      Deliverable:
                      
                      A detailed report documenting your analysis of the critical events, identified IOCs and TTPs, potential impact and risk assessment, recommended actions, and any additional insights or recommendations related to the specific topic of research. This report will serve as a crucial resource for the organization's security decision-making and incident response efforts.
                      
                      Note: 
                      
                      As a Senior Cybersecurity SOC Analyst, your expertise in threat analysis, incident response, and security best practices is essential in ensuring that potential threats are effectively mitigated, and the organization's security posture is continuously strengthened.
                      
                      Ensure that your final answer MUST saved to a markdown."""),
  expected_output=dedent(f"""A detailed report documenting your analysis of the critical events, identified IOCs and TTPs, potential impact and risk assessment, recommended actions, and any additional insights or recommendations related to the specific topic of research. Make sure to validate and double check the severity level assigned to the Triage Specialist"""),
  agent=senior_soc_analyst
)

tk_augument_threat_intell = Task(
  description=dedent(f"""Taking the post created by the writer, take this and save it to a markdown file.
                     Your final answer MUST be a response must be showing that the file was saved ."""),
  expected_output='A saved file name',
  agent=threat_intell
)

tk_write_report = Task(
  description=dedent(f"""Provide comprehensive threat intelligence and analysis to support the organization's cybersecurity efforts. 
                     In this task, you will be generating a detailed threat intelligence report based on the critical events and alerts investigated by the Senior SOC Analyst, 
                     as well as gathering additional insights from various sources.
  
                      Steps:
                        
                        1. Review the detailed report provided by the Senior SOC Analyst, which documents the analysis of critical events, identified IOCs and TTPs, potential impact and risk assessment, and recommended actions.
                        2. Conduct thorough research and gather intelligence from various sources, including:
                          a. Threat intelligence feeds and repositories (e.g., MISP, OTX, VirusTotal)
                          b. Security research publications and industry reports
                          c. Dark web forums and online communities
                          d. Open-source intelligence (OSINT) sources
                          e. Web searches and online resources
                        
                        3. For each critical event or alert, perform the following:
                          a. Correlate the identified IOCs and TTPs with known threat actor groups, campaigns, or malware families.
                          b. Analyze the potential motivations, capabilities, and historical operations of the associated threat actors.
                          c. Assess the prevalence and potential impact of the identified vulnerabilities, if applicable.
                          d. Evaluate the relevance and applicability of published mitigation strategies, detection rules, or security advisories.

                        4. Leverage industry-recognized frameworks like MITRE ATT&CK and OWASP Threat Modeling to contextualize the observed TTPs and vulnerabilities within a broader attack lifecycle or application security context.
                        5. Compile your findings and intelligence into a comprehensive threat intelligence report, following a well-structured format that includes:
                          a. Executive summary
                          b. Overview of critical events and alerts
                          c. Threat actor profiles and motivations
                          d. Detailed analysis of IOCs, TTPs, and vulnerabilities
                          e. Mitigation strategies and recommended actions
                          f. Emerging threats and trends relevant to the organization

                        6. Incorporate visualizations, graphs, or diagrams to enhance the clarity and understanding of the report's content.
                        7. Collaborate with the Senior SOC Analyst and other security teams to validate and refine your findings and recommendations.
                        8. Present and discuss the threat intelligence report with relevant stakeholders, such as the SOC Manager, security leadership, and other cross-functional teams, as necessary.

                      Deliverable:
                      
                      A comprehensive threat intelligence report that provides in-depth analysis and actionable intelligence related to the critical events and alerts investigated by the Senior SOC Analyst. The report should leverage various threat intelligence sources, incorporate industry-recognized frameworks, and provide valuable insights to support the organization's security decision-making and incident response efforts.

                      Note: As a Threat Intelligence, Threat Hunting, and Vulnerability Expert, your ability to synthesize information from diverse sources, analyze complex threats, 
                      and communicate findings effectively is crucial in enhancing the organization's overall security posture and preparedness against emerging threats.
                      """),
  expected_output=dedent(f"""A comprehensive review of the alerts investigated by the Senior SOC Analyst and generate threat intelligence report that provides in-depth analysis and actionable intelligence related to the critical events and alerts. 
                         The report should leverage various threat intelligence sources, incorporate industry-recognized frameworks, 
                         and provide valuable insights to support the organization's security decision-making and incident response efforts."""),
  agent=threat_intell,
  tools=[search_tool],
)

In [24]:
cyber_crew = Crew(
    agents=[soc_manager,
            triage_specialist,
            senior_soc_analyst,
            threat_intell],
    tasks=[tk_thehive_input,
           tk_get_alerts,
           tk_review_alert,
           tk_write_report],
    verbose=2,
    process=Process.hierarchical,
    full_output=True,
    share_crew=False,
    manager_llm=ollama_model,
    max_iter=15,
    step_callback=lambda x: print_agent_output(x,"MasterCrew Agent")
)

2024-04-27 15:07:01,166 - 8448162496 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [25]:
# Kick off the crew's work
results = cyber_crew.kickoff()

# Print the results
print("CyberCrew Work Results:")
print(results)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: USE THE AVAILABLE TOOL and verify alerts for triage and analysis.
                        Go to the TheHive Platform and grab the alerts for triage. Use the following format:
                        <action> <keyword_arguments_as_json>
                        For example:

                        query = "query_alerts {"severity": 3}"
                        alerts = thehive_alerts._run(query)

                        print(alerts)


> Entering new CrewAgentExecutor chain...
Thought: I need to delegate work to co-workers who are qualified for triage analysis from TheHive Platform.
Action: Delegate work to co-worker
Action Input: {"coworker": "Triage Specialist", "task": "Query the Hive Alerts API with severity level 3 alerts and retrieve them in JSON format.", "context": "The Hive platform has a RESTful API that allows querying for alerts. The endpoint to fetch alerts is 'api/alerts'. To filter by severity, include the param

KeyboardInterrupt: 